In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from scipy.stats import ttest_ind, pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import GroupKFold
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.exceptions import UndefinedMetricWarning
import warnings

In [ ]:
file_path = "/content/drive/MyDrive/IT/MScThesis/pilot_VIRDIS/cleaned_virdis_data.csv"
df = pd.read_csv(file_path)

In [ ]:
#@title ADDED CROSS VALIDATION, Concatenating GSR & unpleasantness ratings and performing LR & SVM & RF & XGBoost. Label is bipolar/healthy.

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

df = pd.read_csv("/content/drive/MyDrive/IT/MScThesis/pilot_VIRDIS/cleaned_virdis_data.csv")
group_column = "Group (1=healthy control, 2=bipolar disorder)"
subject_column = "ID"

# Select features
gsr_columns = [col for col in df.columns if "GSR_mean" in col]
unpleasantness_columns = [col for col in df.columns if "unpleasentness_self_rating" in col]
features = gsr_columns + unpleasantness_columns

# Clean dataset
df_clean = df.dropna(subset=features + [group_column, subject_column])
X = df_clean[features]
y = df_clean[group_column]
groups = df_clean[subject_column]

# Define models
models = {
    'Logistic Regression': LogisticRegression(class_weight='balanced'),
    'SVM': SVC(probability=True, kernel='rbf', class_weight='balanced', random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=42)
}

# Cross-validation
cv = GroupKFold(n_splits=5)
results = []

for model_name, model in models.items():
    acc, prec, rec, f1, auc = [], [], [], [], []

    for train_idx, test_idx in cv.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Map to binary labels (0 = healthy, 1 = bipolar)
        y_train_bin = y_train.replace({1: 0, 2: 1})
        y_test_bin = y_test.replace({1: 0, 2: 1})

        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Train model
        model.fit(X_train_scaled, y_train_bin)

        # Predict
        y_pred = model.predict(X_test_scaled)
        y_prob = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, "predict_proba") else y_pred

        # Evaluate
        acc.append(accuracy_score(y_test_bin, y_pred))
        prec.append(precision_score(y_test_bin, y_pred, zero_division=0))
        rec.append(recall_score(y_test_bin, y_pred, zero_division=0))
        f1.append(f1_score(y_test_bin, y_pred, zero_division=0))
        try:
            auc.append(roc_auc_score(y_test_bin, y_prob))
        except ValueError:
            auc.append(np.nan)

    results.append([
        model_name,
        np.mean(acc),
        np.mean(prec),
        np.mean(rec),
        np.mean(f1),
        np.nanmean(auc)
    ])

# Format results
evaluation_results = pd.DataFrame(results, columns=[
    'Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC'
])
print("Cross-Validation Results (Subject-Independent):\n", evaluation_results)

Cross-Validation Results (Subject-Independent):
                  Model  Accuracy  Precision    Recall  F1 Score   ROC AUC
0  Logistic Regression  0.568889   0.620000  0.593333  0.601010  0.615000
1                  SVM  0.475556   0.545000  0.593333  0.557887  0.600000
2        Random Forest  0.542222   0.653571  0.693333  0.634066  0.452778
3              XGBoost  0.520000   0.658333  0.580000  0.561039  0.437778


In [ ]:
#@title QUESTION: IS THERE A DIFFERENCE ACROSS THE GROUPS RELATED TO THE RELATIONSHIP BETWEEN GSR SCORES AND UNPLEASANTNESS RATINGS?
scenario_gsr_mapping = {
    "Neutral_street_scenario_unpleasentness_self_rating": "Neutral_street_GSR_mean",
    "Cantina_scenario_unpleasentness_self_rating": "Cantina_GSR_mean",
    "Elevator_scenario_unpleasentness_self_rating": "Elevator_GSR_mean",
    "Crying_baby_scenario_unpleasentness_self_rating": "Crying_baby_GSR_mean",
    "Presentation_unpleasentness_view_condition_self_rating": "Presentation_view_condition_GSR_mean",
    "Presentation_unpleasentness_dampen_condition_self_rating": "Presentation_dampen_condition_GSR_mean"
}

for gsr_col in scenario_gsr_mapping.values():
    df[gsr_col] = pd.to_numeric(df[gsr_col], errors='coerce')

results = []

for scenario, gsr_col in scenario_gsr_mapping.items():
    df_healthy = df[df["Group (1=healthy control, 2=bipolar disorder)"] == 1]
    df_bipolar = df[df["Group (1=healthy control, 2=bipolar disorder)"] == 2]

    X_healthy = df_healthy[[gsr_col]].dropna()
    y_healthy = df_healthy[scenario].loc[X_healthy.index]
    X_healthy = sm.add_constant(X_healthy)

    X_bipolar = df_bipolar[[gsr_col]].dropna()
    y_bipolar = df_bipolar[scenario].loc[X_bipolar.index]
    X_bipolar = sm.add_constant(X_bipolar)

    coef_healthy = coef_bipolar = se_healthy = se_bipolar = t_stat = p_value = None
    r2_healthy = r2_bipolar = adj_r2_healthy = adj_r2_bipolar = None

    try:
        model_healthy = sm.OLS(y_healthy, X_healthy).fit()
        model_bipolar = sm.OLS(y_bipolar, X_bipolar).fit()

        coef_healthy = model_healthy.params[gsr_col]
        coef_bipolar = model_bipolar.params[gsr_col]
        se_healthy = model_healthy.bse[gsr_col]
        se_bipolar = model_bipolar.bse[gsr_col]

        r2_healthy = model_healthy.rsquared
        r2_bipolar = model_bipolar.rsquared
        adj_r2_healthy = model_healthy.rsquared_adj
        adj_r2_bipolar = model_bipolar.rsquared_adj

        pooled_se = (se_healthy**2 + se_bipolar**2) ** 0.5
        t_stat = (coef_bipolar - coef_healthy) / pooled_se
        df_degrees = len(y_healthy) + len(y_bipolar) - 2
        p_value = stats.t.sf(abs(t_stat), df_degrees) * 2

    except Exception as e:
        print(f"Regression failed for '{scenario}': {e}")

    display_name = scenario.replace("_scenario_unpleasentness_self_rating", "")
    display_name = display_name.replace("_unpleasentness_view_condition_self_rating", " (View)")
    display_name = display_name.replace("_unpleasentness_dampen_condition_self_rating", " (Dampen)")
    display_name = display_name.replace("_", " ").title()

    results.append({
        "Scenario": display_name,
        "GSR Measure": gsr_col.replace("_GSR_mean", "").replace("_", " ").title(),
        "Healthy Coefficient": round(coef_healthy, 3) if coef_healthy is not None else None,
        "Bipolar Coefficient": round(coef_bipolar, 3) if coef_bipolar is not None else None,
        "Difference (Bipolar - Healthy)": round(coef_bipolar - coef_healthy, 3) if coef_healthy is not None and coef_bipolar is not None else None,
        "T-statistic": round(t_stat, 3) if t_stat is not None else None,
        "P-value": round(p_value, 3) if p_value is not None else None,
        "Significance": (
            "***" if p_value is not None and p_value < 0.01 else
            "**" if p_value is not None and p_value < 0.05 else
            "*" if p_value is not None and p_value < 0.1 else ""
        ),
        "R² Healthy": round(r2_healthy, 3) if r2_healthy is not None else None,
        "R² Bipolar": round(r2_bipolar, 3) if r2_bipolar is not None else None,
        "Adj. R² Healthy": round(adj_r2_healthy, 3) if adj_r2_healthy is not None else None,
        "Adj. R² Bipolar": round(adj_r2_bipolar, 3) if adj_r2_bipolar is not None else None
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

             Scenario                   GSR Measure  Healthy Coefficient  Bipolar Coefficient  Difference (Bipolar - Healthy)  T-statistic  P-value Significance  R² Healthy  R² Bipolar  Adj. R² Healthy  Adj. R² Bipolar
       Neutral Street                Neutral Street             -107.518             -280.189                        -172.670       -0.421    0.675                    0.031       0.021           -0.026           -0.018
              Cantina                       Cantina               49.188             -644.860                        -694.048       -1.230    0.225                    0.002       0.068           -0.057            0.031
             Elevator                      Elevator                1.599              596.064                         594.465        1.453    0.153                    0.000       0.109           -0.059            0.074
          Crying Baby                   Crying Baby              -60.493               -7.322                          53.17

In [ ]:
#@title check for normal distribution using #shapiro then use #Mann–Whitney U when needed
from scipy.stats import shapiro, mannwhitneyu

scenario_gsr_mapping = {
    "Neutral_street_scenario_unpleasentness_self_rating": "Neutral_street_GSR_mean",
    "Cantina_scenario_unpleasentness_self_rating": "Cantina_GSR_mean",
    "Elevator_scenario_unpleasentness_self_rating": "Elevator_GSR_mean",
    "Crying_baby_scenario_unpleasentness_self_rating": "Crying_baby_GSR_mean",
    "Presentation_unpleasentness_view_condition_self_rating": "Presentation_view_condition_GSR_mean",
    "Presentation_unpleasentness_dampen_condition_self_rating": "Presentation_dampen_condition_GSR_mean"
}

for gsr_col in scenario_gsr_mapping.values():
    df[gsr_col] = pd.to_numeric(df[gsr_col], errors='coerce')

results = []

for scenario, gsr_col in scenario_gsr_mapping.items():
    df_healthy = df[df["Group (1=healthy control, 2=bipolar disorder)"] == 1]
    df_bipolar = df[df["Group (1=healthy control, 2=bipolar disorder)"] == 2]

    X_healthy = df_healthy[[gsr_col]].dropna()
    y_healthy = df_healthy[scenario].loc[X_healthy.index]
    X_healthy = sm.add_constant(X_healthy)

    X_bipolar = df_bipolar[[gsr_col]].dropna()
    y_bipolar = df_bipolar[scenario].loc[X_bipolar.index]
    X_bipolar = sm.add_constant(X_bipolar)

    coef_healthy = coef_bipolar = se_healthy = se_bipolar = t_stat = p_value = None
    fallback_used = False

    try:
        model_healthy = sm.OLS(y_healthy, X_healthy).fit()
        model_bipolar = sm.OLS(y_bipolar, X_bipolar).fit()

        coef_healthy = model_healthy.params[gsr_col]
        coef_bipolar = model_bipolar.params[gsr_col]
        se_healthy = model_healthy.bse[gsr_col]
        se_bipolar = model_bipolar.bse[gsr_col]

        # Shapiro-Wilk normality test
        shapiro_p_healthy = shapiro(model_healthy.resid).pvalue
        shapiro_p_bipolar = shapiro(model_bipolar.resid).pvalue

        # Check if both residuals are normal
        normal_healthy = shapiro_p_healthy > 0.05
        normal_bipolar = shapiro_p_bipolar > 0.05

        if normal_healthy and normal_bipolar:
            pooled_se = (se_healthy**2 + se_bipolar**2) ** 0.5
            t_stat = (coef_bipolar - coef_healthy) / pooled_se
            df_degrees = len(y_healthy) + len(y_bipolar) - 2
            p_value = stats.t.sf(abs(t_stat), df_degrees) * 2
        else:
            # Non-parametric : Mann–Whitney U
            fallback_used = True
            u_stat, p_value = mannwhitneyu(
                df_bipolar[scenario].dropna(),
                df_healthy[scenario].dropna(),
                alternative='two-sided'
            )
            t_stat = u_stat

    except Exception as e:
        print(f"⚠️ Regression failed for '{scenario}': {e}")


    display_name = scenario.replace("_scenario_unpleasentness_self_rating", "")
    display_name = display_name.replace("_unpleasentness_view_condition_self_rating", " (View)")
    display_name = display_name.replace("_unpleasentness_dampen_condition_self_rating", " (Dampen)")
    display_name = display_name.replace("_", " ").title()

    results.append({
        "Scenario": display_name,
        "GSR Measure": gsr_col.replace("_GSR_mean", "").replace("_", " ").title(),
        "Healthy Coefficient": round(coef_healthy, 3) if coef_healthy is not None else None,
        "Bipolar Coefficient": round(coef_bipolar, 3) if coef_bipolar is not None else None,
        "Difference (Bipolar - Healthy)": round(coef_bipolar - coef_healthy, 3) if coef_healthy is not None and coef_bipolar is not None else None,
        "Stat Type": "T-test" if not fallback_used else "Mann–Whitney U",
        "Test Statistic": round(t_stat, 3) if t_stat is not None else None,
        "P-value": round(p_value, 3) if p_value is not None else None,
        "Significance": (
            "***" if p_value is not None and p_value < 0.01 else
            "**" if p_value is not None and p_value < 0.05 else
            "*" if p_value is not None and p_value < 0.1 else ""
        )
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))
